In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
key_id='YOUR_KEY_ID'
access_key='YOUR_ACCESS_KEY'
region="us-east-2"
sc.setSystemProperty("com.amazonaws.services.s3.enableV4","true")
sc._jsc.hadoopConfiguration().set("fs.s3a.awsAccessKeyId",key_id)
sc._jsc.hadoopConfiguration().set("fs.s3a.awsSecretAccessKey",access_key)
sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint","s3-"+region+".amazonaws.com")

In [3]:
#We have 3 datasets we have are:
#Complete dataset : The one with 26million+ ratings but without any user data except their user_id
#Small dataset : Which is a subset of Complete dataset and consists a total of 100,000 ratings of 9,000 movies by 700 users
#User dataset : 1 million ratings from 6000 users on 4000 movies and consists of user data 'user_id', 'gender', 'age', 'occupation' and 'zip_code'
complete_dataset_path = "PATH/TO/COMPLETE/DATASET"
small_dataset_path = "PATH/TO/SMALL/DATASET"
user_dataset_path = "PATH/TO/USER/DATASET"

In [4]:
#Loading the complete ratings.csv file into a RDD, for preprocessing, and removing the header from it.
complete_ratings_file = complete_dataset_path+'ratings.csv'
complete_ratings_raw_data = sc.textFile(complete_ratings_file)
complete_ratings_raw_data_header = complete_ratings_raw_data.take(1)[0]
complete_ratings_raw_data = complete_ratings_raw_data.filter(lambda line: line!=complete_ratings_raw_data_header)

In [5]:
#Data read into the RDD from ratings.csv file of complete dataset
complete_ratings_raw_data.take(10)

['1,110,1.0,1425941529',
 '1,147,4.5,1425942435',
 '1,858,5.0,1425941523',
 '1,1221,5.0,1425941546',
 '1,1246,5.0,1425941556',
 '1,1968,4.0,1425942148',
 '1,2762,4.5,1425941300',
 '1,2918,5.0,1425941593',
 '1,2959,4.0,1425941601',
 '1,4226,4.0,1425942228']

In [6]:
#Splitting the data of ratings.csv in 4 different columns('user_id', 'movie_id', 'rating', 'timestamp') with ,(comma) being the delimiter
#converting epoch time to "yyyy-mm-dd HH24:MM:SS" format
import time
complete_ratings_raw_data=complete_ratings_raw_data.map(lambda x: x.split(',')[0]+'%'+x.split(',')[1]+'%'+\
                                                                   x.split(',')[2]+'%'+time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(int(x.split(',')[3]))))
complete_ratings_raw_data.take(10)

['1%110%1.0%2015-03-09 22:52:09',
 '1%147%4.5%2015-03-09 23:07:15',
 '1%858%5.0%2015-03-09 22:52:03',
 '1%1221%5.0%2015-03-09 22:52:26',
 '1%1246%5.0%2015-03-09 22:52:36',
 '1%1968%4.0%2015-03-09 23:02:28',
 '1%2762%4.5%2015-03-09 22:48:20',
 '1%2918%5.0%2015-03-09 22:53:13',
 '1%2959%4.0%2015-03-09 22:53:21',
 '1%4226%4.0%2015-03-09 23:03:48']

In [7]:
#Saving the Preprocessed complete ratings data in a CSV in the local directory.
complete_ratings_raw_data.coalesce(1).saveAsTextFile("complete_ratings_processed_data.csv")

In [8]:
#Loading the complete movies.csv file into a RDD, for preprocessing, and removing the header from it.
complete_movies_file = complete_dataset_path+'movies.csv'
complete_movies_raw_data = sc.textFile(complete_movies_file)
complete_movies_raw_data_header = complete_movies_raw_data.take(1)[0]
complete_movies_raw_data = complete_movies_raw_data.filter(lambda line: line!=complete_movies_raw_data_header)

In [9]:
#Data read into the RDD from movies.csv file of complete dataset
complete_movies_raw_data.take(10)

['1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy',
 '2,Jumanji (1995),Adventure|Children|Fantasy',
 '3,Grumpier Old Men (1995),Comedy|Romance',
 '4,Waiting to Exhale (1995),Comedy|Drama|Romance',
 '5,Father of the Bride Part II (1995),Comedy',
 '6,Heat (1995),Action|Crime|Thriller',
 '7,Sabrina (1995),Comedy|Romance',
 '8,Tom and Huck (1995),Adventure|Children',
 '9,Sudden Death (1995),Action',
 '10,GoldenEye (1995),Action|Adventure|Thriller']

In [10]:
#Splitting the data of movies.csv in 3 different columns('movie_id', 'title'', 'genres') with ,(comma) being the delimiter
complete_movies_raw_data=complete_movies_raw_data.map(lambda x: [x.split(',')[0], ','.join(x.split(',')[1:-1]),\
                                                                   x.split(',')[-1]])
complete_movies_raw_data.take(10)

[['1', 'Toy Story (1995)', 'Adventure|Animation|Children|Comedy|Fantasy'],
 ['2', 'Jumanji (1995)', 'Adventure|Children|Fantasy'],
 ['3', 'Grumpier Old Men (1995)', 'Comedy|Romance'],
 ['4', 'Waiting to Exhale (1995)', 'Comedy|Drama|Romance'],
 ['5', 'Father of the Bride Part II (1995)', 'Comedy'],
 ['6', 'Heat (1995)', 'Action|Crime|Thriller'],
 ['7', 'Sabrina (1995)', 'Comedy|Romance'],
 ['8', 'Tom and Huck (1995)', 'Adventure|Children'],
 ['9', 'Sudden Death (1995)', 'Action'],
 ['10', 'GoldenEye (1995)', 'Action|Adventure|Thriller']]

In [11]:
#Removing the release year of the movie in the 'title' into a different column 'year'
complete_movies_raw_data=complete_movies_raw_data.map(lambda x: x[0]+'::'+'('.join(x[1].split('(')[:-1]).replace('\"','')+'::'+ \
                                                                 x[1].split('(')[-1].replace(')','').replace('\"','')+'::'+\
                                                                 x[2])
complete_movies_raw_data.take(20)

['1::Toy Story ::1995::Adventure|Animation|Children|Comedy|Fantasy',
 '2::Jumanji ::1995::Adventure|Children|Fantasy',
 '3::Grumpier Old Men ::1995::Comedy|Romance',
 '4::Waiting to Exhale ::1995::Comedy|Drama|Romance',
 '5::Father of the Bride Part II ::1995::Comedy',
 '6::Heat ::1995::Action|Crime|Thriller',
 '7::Sabrina ::1995::Comedy|Romance',
 '8::Tom and Huck ::1995::Adventure|Children',
 '9::Sudden Death ::1995::Action',
 '10::GoldenEye ::1995::Action|Adventure|Thriller',
 '11::American President, The ::1995::Comedy|Drama|Romance',
 '12::Dracula: Dead and Loving It ::1995::Comedy|Horror',
 '13::Balto ::1995::Adventure|Animation|Children',
 '14::Nixon ::1995::Drama',
 '15::Cutthroat Island ::1995::Action|Adventure|Romance',
 '16::Casino ::1995::Crime|Drama',
 '17::Sense and Sensibility ::1995::Drama|Romance',
 '18::Four Rooms ::1995::Comedy',
 '19::Ace Ventura: When Nature Calls ::1995::Comedy',
 '20::Money Train ::1995::Action|Comedy|Crime|Drama|Thriller']

In [12]:
#Saving the Preprocessed complete movies data in a CSV in the local directory.
complete_movies_raw_data.coalesce(1).saveAsTextFile("complete_movies_processed_data.csv")